In [99]:
#import all pakages
import pandas as pd



In [100]:
#import the input file
input_xl= pd.read_excel(r'C:\Users\HP\OneDrive\Documents\GitHub\monthly-report-generator\data\input.xlsx', header=14, usecols="B:AC")
print(input_xl.head())
print(type(input_xl))

ref= pd.read_excel(r'C:\Users\HP\OneDrive\Documents\GitHub\monthly-report-generator\data\ref.xlsx')
print(ref.head())


   Zone           CIRCLE          BA         SSA Device Role       IPAddress  \
0  EAST  ANDAMAN NICOBAR  PORT BLAIR  PORT BLAIR     TIP-OLT   10.211.223.30   
1  EAST  ANDAMAN NICOBAR  PORT BLAIR  PORT BLAIR     TIP-OLT    10.211.223.9   
2  EAST  ANDAMAN NICOBAR  PORT BLAIR  PORT BLAIR     TIP-OLT  10.215.223.116   
3  EAST  ANDAMAN NICOBAR  PORT BLAIR  PORT BLAIR     TIP-OLT  10.215.223.154   
4  EAST  ANDAMAN NICOBAR  PORT BLAIR  PORT BLAIR     TIP-OLT   10.215.223.68   

     VENDOR    Device Name  Carry Forward  Down Count  ...  > 7 Days  \
0       DBC  10.211.223.30            0.0         2.0  ...       0.0   
1   NETLINK   10.211.223.9            0.0         7.0  ...       0.0   
2  Syrotech       epon-olt            0.0         2.0  ...       0.0   
3  Optronix       Luckhnow            0.0         2.0  ...       0.0   
4   Netlink  10.215.223.68            0.0         1.0  ...       0.0   

   Total Resolved  Total Down Duration              MTTR  Pending Faults  \
0         

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


STEP 1: Keep only the necessary rows

In [101]:
step1=input_xl[['IPAddress', 'VENDOR', 'Device Name','Total Down Duration', 'MTTR']]
column_names = input_xl.columns.tolist()
print(column_names)
print(step1.head())


['Zone', 'CIRCLE', 'BA', 'SSA', 'Device Role', 'IPAddress', 'VENDOR', 'Device Name', 'Carry Forward', 'Down Count', 'Total Count', '< 15 Mins', '< 1 Hour', '< 4 Hours', '< 8 Hours', '< 24 Hours', '< 3 Days', '< 7 Days', '> 7 Days', 'Total Resolved', 'Total Down Duration', 'MTTR', 'Pending Faults', 'Pending < 1 Hour', 'Pending < 24 Hours', 'Pending < 3 Days', 'Pending < 7 Days', 'Pending > 7 Days']
        IPAddress    VENDOR    Device Name Total Down Duration  \
0   10.211.223.30       DBC  10.211.223.30  0days 18Hrs 10mins   
1    10.211.223.9   NETLINK   10.211.223.9  0days 15Hrs 39mins   
2  10.215.223.116  Syrotech       epon-olt   0days 5Hrs 35mins   
3  10.215.223.154  Optronix       Luckhnow   0days 5Hrs 35mins   
4   10.215.223.68   Netlink  10.215.223.68   0days 1Hrs 20mins   

               MTTR  
0  0days 9h:04m:55s  
1  0days 2h:14m:11s  
2  0days 2h:47m:26s  
3  0days 2h:47m:28s  
4  0days 1h:20m:03s  


STEP 2: Finding weather the device is located at a rural or urban place

In [102]:
step2 = pd.merge(step1, ref, on='IPAddress')
#step2['MTTR'] = step2['MTTR'].fillna('0days 0h:00m:00s')
step2 = step2.dropna(subset=['MTTR'])


print(step2.head())
#step2['MTTR'] = step2['MTTR'].str.removeprefix('0days ')
#remove rows that have MTTR not starting with '0days'
step2b = step2[step2['MTTR'].str.startswith('0days')]

        IPAddress    VENDOR    Device Name Total Down Duration  \
0   10.211.223.30       DBC  10.211.223.30  0days 18Hrs 10mins   
1    10.211.223.9   NETLINK   10.211.223.9  0days 15Hrs 39mins   
2  10.215.223.116  Syrotech       epon-olt   0days 5Hrs 35mins   
3  10.215.223.154  Optronix       Luckhnow   0days 5Hrs 35mins   
4   10.215.223.68   Netlink  10.215.223.68   0days 1Hrs 20mins   

               MTTR         Place   Type  
0  0days 9h:04m:55s    PROTHRAPUR  RURAL  
1  0days 2h:14m:11s     TUSHNABAD  RURAL  
2  0days 2h:47m:26s     KARMATANG  RURAL  
3  0days 2h:47m:28s    MGGCOLLEGE  RURAL  
4  0days 1h:20m:03s  SWARAJ DWEEP  RURAL  


STEP 3: Separate rural from urban

In [103]:
rural= step2b[step2b['Type'] == 'RURAL']
urban = step2b[step2b['Type'] == 'URBAN']
r=len(rural)
u=len(urban)

STEP 4: urban <4h  and rural<6h

In [104]:
#remove first 6 elements from a string
rural['MTTR'] = rural['MTTR'].str[6:]
urban['MTTR'] = urban['MTTR'].str[6:]

rural = rural[rural['MTTR'].str.startswith(('6h:00m:00s','5h','4h', '3h', '2h', '1h', '0h'))]
urban = urban[urban['MTTR'].str.startswith(('4h:00m:00s', '3h', '2h', '1h', '0h'))]

rural['MTTR'] = rural['MTTR'].str.replace('[hms]', '', regex=True)
urban['MTTR'] = urban['MTTR'].str.replace('[hms]', '', regex=True)

rural['MTTR'] = pd.to_datetime(rural['MTTR'], format='%H:%M:%S').dt.time
urban['MTTR'] = pd.to_datetime(urban['MTTR'], format='%H:%M:%S').dt.time


C:\Users\HP\AppData\Local\Temp\ipykernel_25004\3311858391.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rural['MTTR'] = rural['MTTR'].str[6:]
C:\Users\HP\AppData\Local\Temp\ipykernel_25004\3311858391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  urban['MTTR'] = urban['MTTR'].str[6:]


STEP 5: Add time to find mean MTTR for rural and urban areas

In [105]:
rural['MTTR'] = pd.to_timedelta(rural['MTTR'].astype(str))
average_timer = rural['MTTR'].mean()

urban['MTTR'] = pd.to_timedelta(urban['MTTR'].astype(str))
average_timeu = urban['MTTR'].mean()
print("Average time for rural MTTR: ", average_timer)
print("Average time for Urban MTTR: ", average_timeu)


Average time for rural MTTR:  0 days 01:49:24.485074626
Average time for Urban MTTR:  0 days 01:06:42.600000
